In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import heapq
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Load training, testing, and validation data
train_data = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test_data = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [ ]:
#Data Cleaning
#Drop columns with too many null values
train_data = train_data.drop(columns = ['Alley', 'PoolQC','Fence','MiscFeature'])
test_data = test_data.drop(columns = ['Alley', 'PoolQC','Fence','MiscFeature'])
#Classify into categorical and numerical cols
numerical_cols = list(filter(lambda name : ((train_data[name].dtype in ['int64','float64']) and (len(train_data[name].value_counts()) > 30) ), train_data.columns))
numerical_cols = numerical_cols[1:-1] #Take out SalePrice
categorical_cols = []
for col in train_data.columns:
    if col != 'SalePrice' and col != 'Id' and (col not in numerical_cols):
        categorical_cols.append(col)

In [ ]:
for col in numerical_cols:
    train_data[col] = train_data[col].astype(np.float64)
    train_data[col].fillna(train_data[col].mean(),inplace = True)
    test_data[col] = test_data[col].astype(np.float64)
    test_data[col].fillna(test_data[col].mean(),inplace = True)

In [ ]:
#Look at the correlations among all the numerical columns
train_data_corr = train_data[numerical_cols + ['SalePrice']].corr()
train_data_corr.sort_values(by = 'SalePrice', inplace = True)
train_data_corr = train_data_corr['SalePrice']
#print(train_data_corr)

#print(numerical_cols)
colname = 'ScreenPorch'

train_data.sort_values(inplace = True, by = colname)
#plt.scatter(train_data[colname], train_data['SalePrice'])
#plt.show()

In [ ]:
#EnclosedPorch: drop
#BsmtFinSF2: drop
#ScreenPorch: Quadratic wout 0's
#BsmtUnfSF: Linear in 1000+
#LotArea: Drop
#OpenPorchSF: Drop
#2ndFlrSF: Drop
#WoodDeckSF: Linear wout 0's
#LotFrontage: Linear
#BsmtFinSF1: Linear wout 0's
#GarageYrBlt: Linear in 1980+
#MasVnrArea: Linear wout 0s
#YearRemodAdd: Linear in 1970+
#YearBuilt: Linear in 1960+
#1stFlrSF: linear
#TotalBsmtSF: linear wout 0's
#GarageArea: linear wout 0's
#GrLivArea: linear

In [ ]:
drop_cols = ['EnclosedPorch','BsmtFinSF2','LotArea','OpenPorchSF','2ndFlrSF']
train_data.drop(columns = drop_cols)
test_data.drop(columns = drop_cols)

def indicatormod(val):
    if val < 0.0001:
        return 1.0
    else:
        return 0.0

def threshmodbelow(val, thresh):
    if val < thresh:
        return val
    else:
        return 0.0

def threshmodabove(val, thresh):
    if val >= thresh:
        return val
    else:
        return 0.0
    


#ScreenPorch
train_data['ScreenPorch^2'] = train_data['ScreenPorch']**2
test_data['ScreenPorch^2'] = test_data['ScreenPorch']**2
train_data['ScreenPorch0I'] = train_data['ScreenPorch'].apply(indicatormod)
test_data['ScreenPorch0I'] = test_data['ScreenPorch'].apply(indicatormod)
numerical_cols.append('ScreenPorch^2')
numerical_cols.append('ScreenPorch0I')

#BsmtFinSF2
train_data['BsmtUnfSFPre'] = train_data['BsmtUnfSF'].apply(lambda x : threshmodbelow(x, 1000))
train_data['BsmtUnfSFPost'] = train_data['BsmtUnfSF'].apply(lambda x : threshmodabove(x, 1000))
test_data['BsmtUnfSFPre'] = test_data['BsmtUnfSF'].apply(lambda x : threshmodbelow(x, 1000))
test_data['BsmtUnfSFPost'] = test_data['BsmtUnfSF'].apply(lambda x : threshmodabove(x, 1000))
numerical_cols.remove('BsmtUnfSF')
numerical_cols.append('BsmtUnfSFPre')
numerical_cols.append('BsmtUnfSFPost')

#WoodDeckSF
train_data['WoodDeckSF0I'] = train_data['WoodDeckSF'].apply(indicatormod)
test_data['WoodDeckSF0I'] = test_data['WoodDeckSF'].apply(indicatormod)
numerical_cols.append('WoodDeckSF0I')

#BsmtFinSF1
train_data['BsmtFinSF10I'] = train_data['BsmtFinSF1'].apply(indicatormod)
test_data['BsmtFinSF10I'] = test_data['BsmtFinSF1'].apply(indicatormod)
numerical_cols.append('BsmtFinSF10I')

#GarageYrBlt
train_data['GarageYrBltPre'] = train_data['GarageYrBlt'].apply(lambda x : threshmodbelow(x, 1980))
train_data['GarageYrBltPost'] = train_data['GarageYrBlt'].apply(lambda x : threshmodabove(x, 1980))
test_data['GarageYrBltPre'] = test_data['GarageYrBlt'].apply(lambda x : threshmodbelow(x, 1980))
test_data['GarageYrBltPost'] = test_data['GarageYrBlt'].apply(lambda x : threshmodabove(x, 1980))
numerical_cols.remove('GarageYrBlt')
numerical_cols.append('GarageYrBltPre')
numerical_cols.append('GarageYrBltPost')

#MasVnrArea
train_data['MasVnrArea0I'] = train_data['MasVnrArea'].apply(indicatormod)
test_data['MasVnrArea0I'] = test_data['MasVnrArea'].apply(indicatormod)
numerical_cols.append('MasVnrArea0I')

#YearRemodAdd
train_data['YearRemodAddPre'] = train_data['YearRemodAdd'].apply(lambda x : threshmodbelow(x, 1970))
train_data['YearRemodAddPost'] = train_data['YearRemodAdd'].apply(lambda x : threshmodabove(x, 1970))
test_data['YearRemodAddPre'] = test_data['YearRemodAdd'].apply(lambda x : threshmodbelow(x, 1970))
test_data['YearRemodAddPost'] = test_data['YearRemodAdd'].apply(lambda x : threshmodabove(x, 1970))
numerical_cols.remove('YearRemodAdd')
numerical_cols.append('YearRemodAddPre')
numerical_cols.append('YearRemodAddPost')

#YearBuilt
train_data['YearBuiltPre'] = train_data['YearBuilt'].apply(lambda x : threshmodbelow(x, 1960))
train_data['YearBuiltPost'] = train_data['YearBuilt'].apply(lambda x : threshmodabove(x, 1960))
test_data['YearBuiltPre'] = test_data['YearBuilt'].apply(lambda x : threshmodbelow(x, 1960))
test_data['YearBuiltPost'] = test_data['YearBuilt'].apply(lambda x : threshmodabove(x, 1960))
numerical_cols.remove('YearBuilt')
numerical_cols.append('YearBuiltPre')
numerical_cols.append('YearBuiltPost')

#GarageArea
train_data['GarageArea0I'] = train_data['GarageArea'].apply(indicatormod)
test_data['GarageArea0I'] = test_data['GarageArea'].apply(indicatormod)
numerical_cols.append('GarageArea0I')

#TotalBsmtSF
train_data['TotalBsmtSF0I'] = train_data['TotalBsmtSF'].apply(indicatormod)
test_data['TotalBsmtSF0I'] = test_data['TotalBsmtSF'].apply(indicatormod)
numerical_cols.append('TotalBsmtSF0I')


In [ ]:
X = train_data[numerical_cols]
model = LinearRegression().fit(X, train_data['SalePrice']) 
train_data['Residuals'] = np.abs(model.predict(X) - train_data['SalePrice'])
#print(model.score(X, train_data['SalePrice']))

train_data_new = train_data[train_data['Residuals'] < 70000]
X_new = train_data_new[numerical_cols]
model = LinearRegression().fit(X_new, train_data_new['SalePrice'])
train_data_new = train_data_new.drop(columns = ['Residuals'])
train_data_new['Residuals'] = model.predict(X_new) - train_data_new['SalePrice']




In [ ]:
def numericalize(col_name, df):
    set_of_values = set()
    for value in df[col_name].values:
        set_of_values.add(value)
    index = 0
    my_dict = {}
    for value in set_of_values:
        my_dict[value] = index
        index = index+1
    return df[col_name].replace(to_replace = my_dict)

categorical_cols_num = []
for col in categorical_cols:
    categorical_cols_num.append(col + '_num')
    train_data_new[col + '_num'] = numericalize(col, train_data_new)
    test_data[col + '_num'] = numericalize(col, test_data)

X_new = train_data_new[categorical_cols_num]
add_model = RandomForestRegressor(n_estimators=40,max_depth=8, random_state = 10)
add_model.fit(X_new, train_data_new['SalePrice'])
print(add_model.score(X_new, train_data_new['SalePrice']))
test_data['Predicted_Prices'] = (model.predict(test_data[numerical_cols]) + add_model.predict(test_data[categorical_cols_num]))/2.0

In [ ]:
train_data_new[categorical_cols[0]].values

**Plan**
Step 1. Check by hand each column to see if it is numerical or categorical X  
Step 2. Use median instead of mean for the KNN X  
Step 3. Go through each column and visualize to determine what modification of column to put into the linear regression X  
Step 4. Run linear regression, remove outliers, and then run linear regression again X  
Step 5. Use weighted KNN separately on the categorical columns and find optimal hyperparameters for the categorical weighted KNN based on impurity  
Step 6. Drop lowest weighted categorical columns to get a cleaner regression  
Step 7. Go through each categorical column to see if you can divide weights to make them piecewise different  
Step 8. Optimize alpha-beta weighting between categorical KNN and linear regression  

In [ ]:
ans = test_data[['Id', 'Predicted_Prices']].to_numpy()
print(ans)
ls1 = []
ls2 = []
for i in range(len(ans)):
    ls1.append(int(ans[i][0]))
    ls2.append(ans[i][1])
dff_dict = {'Id' : ls1, 'SalePrice' : ls2}
dff = pd.DataFrame(dff_dict)
dff.to_csv('tuned_output.csv')